# EXPERIMENT : MINING CANDLESTICK USING PERMUTATION ENTROPY
In this experiment, we would try to expand the pattern probabilistic effects experminet from Tradingview (`Significant Move` Indicator).

Procedure: 
- Encode for all possible arrangements of the OHLC values. Filter out illogical, or redundant possibilities

In [1]:
# Import Libraries
import itertools

In [2]:
# Define the possible components of the comparison rules
series_options = ['open', 'high', 'low', 'close']
comparisons = ['>', '<']

# Function to generate all possible comparison rules
def generate_comparison_rules(maximum_lag):
    lags = list(range(maximum_lag))
    rules = []
    for series1, series2, lag1, lag2, comparison in itertools.product(series_options, series_options, lags, lags, comparisons):
        if series1 != series2 or lag1 != lag2:
            rule = f"{series1}[{lag1}] {comparison} {series2}[{lag2}]"
            rules.append(rule)
    return rules

# Generate all possible patterns with a given pattern size
def generate_all_patterns(pattern_size, maximum_lag):
    comparison_rules = generate_comparison_rules(maximum_lag)
    all_patterns = list(itertools.combinations(comparison_rules, pattern_size))
    return all_patterns

# Function to check for logical consistency in a pattern
def is_logical(pattern, exclude_constraints:list[str]):
    rules_checked = []
    fixed_series = ['high', 'low']
    
    for rule in pattern:
        left, comp, right = rule.split()
        flip_comp = '>' if comp == '<' else '<'

        left_series, left_lag = left[:-3], left[-2]
        right_series, right_lag = right[:-3], right[-2]

        # Check for illogical high/low comparisons
        if (left_series in fixed_series) and (right_series in fixed_series) and (left_lag == right_lag):
            return False

        # Apply constraints
        for constraint in exclude_constraints:
            split_constraint = constraint.replace(' ', '').split('/')
            if any([split_constraint[0] in x for x in [left, right]]) and any([split_constraint[1] in x for x in [left, right]]):
                return False

        # Generate all conditions that would cause the rule to be illogical
        analogs = [
            (left, right, comp),
            (right, left, flip_comp),
            (left, right, flip_comp)
        ]

        # Check for duplicates or contradictory rules
        if not set(rules_checked).isdisjoint(set(analogs)):
            return False
        else:
            rules_checked.extend(analogs)
        
    return True

# Function to filter patterns based on constraints and logical consistency
def filter_patterns(patterns, include_constraints, exclude_constraints):
    def pattern_meets_constraints(pattern):
        for rule in pattern:
            if not any(constraint in rule for constraint in include_constraints):
                return False
            if any(constraint in rule for constraint in exclude_constraints):
                return False
        
        extended_exclude_constraints = [constraint for constraint in exclude_constraints if '/' in constraint]
        return is_logical(pattern, extended_exclude_constraints)
    
    filtered_patterns = [pattern for pattern in patterns if pattern_meets_constraints(pattern)]
    return filtered_patterns


# pattern_size = 3
# max_lag = 2
# all_patterns = generate_all_patterns(pattern_size, max_lag)

In [3]:
# include_constraints = ['high', 'low', 'close']
# exclude_constraints = ['open', 'high/low']

# filtered_patterns = filter_patterns(all_patterns, include_constraints, exclude_constraints)

# len(filtered_patterns)

# CANDLE STICK TARGET STATISTICS

In [4]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import yfinance as yf

In [36]:
# Define ticker symbols
# yf_df : pd.DataFrame = yf.download('GBPUSD=X') # EURUSD
yf_df = yf.download('ES=F') # S&P 500 as a proxy for ES futures

[*********************100%%**********************]  1 of 1 completed


In [37]:
raw_df = yf_df.copy()
raw_df.reset_index(drop=False, inplace=True)
raw_df.columns = raw_df.columns.str.lower()
raw_df.set_index('date', inplace=True)
raw_df.drop(['adj close', 'volume'], axis=1, inplace=True)

raw_df.head()

,open,high,low,close
date,,,,
2000-09-18,1485.25,1489.75,1462.25,1467.5
2000-09-19,1467.00,1482.75,1466.75,1478.5
2000-09-20,1478.75,1480.50,1450.25,1469.5
2000-09-21,1470.25,1474.00,1455.50,1469.5
2000-09-22,1454.75,1471.00,1436.75,1468.5


In [38]:
df = raw_df.copy()
df['hl2'] = np.average([df['high'], df['low']], axis=0)

threshold_strong_move = 1
atr = ta.atr(df['high'], df['low'], df['close'], 14)
doji_atr_limit = atr * 0.05
body_size = np.abs(df['open'] - df['close'])
df_1 = df.shift(1)
df_2 = df.shift(2)

strong_move = body_size >= threshold_strong_move
up_candle = df['open'] < df['close']
dn_candle = df['open'] > df['close']
new_high = df['high'] > df_1['high']
new_low = df['low'] < df_1['low']
df

,open,high,low,close,hl2
date,,,,,
2000-09-18,1485.25,1489.75,1462.25,1467.50,1476.000
2000-09-19,1467.00,1482.75,1466.75,1478.50,1474.750
2000-09-20,1478.75,1480.50,1450.25,1469.50,1465.375
2000-09-21,1470.25,1474.00,1455.50,1469.50,1464.750
2000-09-22,1454.75,1471.00,1436.75,1468.50,1453.875
...,...,...,...,...,...
2024-05-16,5331.50,5349.00,5315.50,5320.25,5332.250
2024-05-17,5318.25,5328.75,5305.75,5327.25,5317.250
2024-05-20,5329.25,5348.25,5323.75,5331.75,5336.000


### SPECIFIC CANDLE STICK TARGET STATISTICS

- engulfing = down_candle[1] and up_candle[0] and (close[0] > max(open[1], close[1]))  # Reversal
- englufing_strict_1 = down_candle[1] and up_candle[0] and new_low[1] and (open[0] <= close[0]) and (close[0] > max(open[1], close[1])) and (high[0] > high[1])# Reversal
- englufing_strict_2 = down_candle[1] and up_candle[0] and new_low[1] and (low[0] < low[1]) and (open[0] <= close[0]) and (close[0] > high[1])  # Reversal

- doji = engulfing[1] and (abs(open - close) < threshold) # Reversal
- doji_strict = engulfing[1] and (abs(open - close) < threshold) and (high[0] < high[1]) and (low[0] > low[1]) # Reversal

- piercing = engulfing but with >= 50% engulfing
- piercing_strict = down_candle[1] and strong_candle[1] and (open[0] < [close[0]) and (close[0] > hl2[1])
- piercing_strict = engulfing and down_candle[1] and (open[0] < [close[0]) and (close[0] > hl2[1])
    
- inside = engulfing[1] and (high[0] > high[1]) # Reversal
- kicker = down_candle[1] and (open[0] > close[1]) and (high[0] > high[1])

In [39]:
engulfing = dn_candle.shift(1) & up_candle & (df['close'] > np.maximum(df_1['open'], df_1['close']))  # Reversal
engulfing_strict_1 = dn_candle.shift(1) & up_candle & new_low.shift(1) & (df['open'] <= df_1['close']) & (df['close'] > np.maximum(df_1['open'], df_1['close'])) & (df['high'] > df_1['high']) # Reversal
engulfing_strict_2 = dn_candle.shift(1) & up_candle & new_high.shift(1) & (df['low'] < df_1['low']) & (df['open'] <= df['close']) & (df['close'] > df_1['high'])  # Reversal

doji = engulfing.shift(1) & (body_size < doji_atr_limit) # Reversal
doji_strict = engulfing.shift(1) & (body_size < doji_atr_limit) & (df['high'] < df_1['high']) & (df['low'] > df_1['low']) # Reversal

inside = engulfing.shift(1) & (df['high'] > df_1['high']) # Reversal
kicker = dn_candle.shift(1) & (df['open'] > df_1['close']) & (df['high'] > df_1['high'])


# Piercing pattern definitions
piercing = dn_candle.shift(1) & up_candle & (df['close'] >= df_1['close'] + 0.5 * (df_1['open'] - df_1['close'])) & (df['close'] < df_1['open'])  # >= 50% engulfing
piercing_strict = dn_candle.shift(1) & up_candle & (df['open'] < df_1['close']) & (df['close'] > df_1['hl2'])  # Strong piercing
piercing_strict_2 = engulfing & dn_candle.shift(1) & (df['open'] < df_1['close']) & (df['close'] > df_1['hl2'])  # Engulfing and strong piercing



# BEARS

# Bearish patterns
bearish_engulfing = up_candle.shift(1) & dn_candle & (df['close'] < np.minimum(df_1['open'], df_1['close']))  # Reversal
bearish_engulfing_strict_1 = up_candle.shift(1) & dn_candle & new_high.shift(1) & (df['open'] >= df_1['close']) & (df['close'] < np.minimum(df_1['open'], df_1['close'])) & (df['low'] < df_1['low'])  # Reversal
bearish_engulfing_strict_2 = up_candle.shift(1) & dn_candle & new_low.shift(1) & (df['high'] > df_1['high']) & (df['open'] >= df['close']) & (df['close'] < df_1['low'])  # Reversal

bearish_doji = bearish_engulfing.shift(1) & (body_size < doji_atr_limit)  # Reversal
bearish_doji_strict = bearish_engulfing.shift(1) & (body_size < doji_atr_limit) & (df['high'] < df_1['high']) & (df['low'] > df_1['low'])  # Reversal

bearish_inside = bearish_engulfing.shift(1) & (df['low'] < df_1['low'])  # Reversal
bearish_kicker = up_candle.shift(1) & (df['open'] < df_1['close']) & (df['low'] < df_1['low'])

# Dark cloud cover (bearish analog of piercing)
bearish_piercing = up_candle.shift(1) & dn_candle & (df['close'] <= df_1['close'] - 0.5 * (df_1['close'] - df_1['open'])) & (df['close'] > df_1['open'])  # >= 50% engulfing
bearish_piercing_strict = up_candle.shift(1) & dn_candle & (df['open'] > df_1['close']) & (df['close'] < df_1['hl2'])  # Strong dark cloud
bearish_piercing_strict_2 = bearish_engulfing & up_candle.shift(1) & (df['open'] > df_1['close']) & (df['close'] < df_1['hl2'])  # Engulfing and strong dark cloud


In [40]:
def analyse_pattern(pattern_df:pd.Series, original_df:pd.DataFrame, hold_period:int=1):
    # Possible Targets : 
    # - open[0]
    # - high[0]
    # - low[0]
    # - ohlc[1]

    data = original_df.copy()
    data_forward = data.shift(-hold_period)
    data_backward = data.shift(1)

    data_max_high = data_forward['high'].rolling(hold_period).max()
    data_min_low = data_forward['low'].rolling(hold_period).min()

    data['target_open'] = (data_max_high >= data['open']) & (data_min_low <= data['open']) # Checks if open[0] is hit on the next candle
    data['target_high'] = (data_max_high >= data['high']) & (data_min_low <= data['high']) # Checks if high[0] is hit on the next candle
    data['target_low'] = (data_max_high >= data['low']) & (data_min_low <= data['low']) # Checks if low[0] is hit on the next candle

    data['target_open_1'] = (data_max_high >= data_backward['open']) & (data_min_low <= data_backward['open']) # Checks if previous open is hit on the next candle
    data['target_high_1'] = (data_max_high >= data_backward['high']) & (data_min_low <= data_backward['high']) # Checks if previous high is hit on the next candle
    data['target_low_1'] = (data_max_high >= data_backward['low']) & (data_min_low <= data_backward['low']) # Checks if previous low is hit on the next candle
    data['target_close_1'] = (data_max_high >= data_backward['close']) & (data_min_low <= data_backward['close']) # Checks if previous close is hit on the next candle

    data['pattern'] = pattern_df

    # Get the statistics for each target
    stats = { 
        'target' : [],
        'total' : [],
        'success' : [],
        'probability' : []
    }
    
    for target in ["target_open", "target_high", "target_low", "target_open_1", "target_high_1", "target_low_1", "target_close_1"]:

        # Select rows where pattern is found
        selected = data[data['pattern']]
        count = len(selected)
        success = len(selected[selected[target]])
        probability = round(100 * success / count, 3)

        stats['target'].append(target)
        stats['total'].append(count)
        stats['success'].append(success)
        stats['probability'].append(probability)
        

    return display(pd.DataFrame(stats))
    

def analyse_pattern_multi(pattern_list:list, original_df:pd.DataFrame, hold_period:int=1):
    # Possible Targets : 
    # - open[0]
    # - high[0]
    # - low[0]
    # - ohlc[1]

    data = original_df.copy()
    data_forward = data.shift(-hold_period)
    data_backward = data.shift(1)

    data_max_high = data_forward['high'].rolling(hold_period).max()
    data_min_low = data_forward['low'].rolling(hold_period).min()

    data['target_open'] = (data_max_high >= data['open']) & (data_min_low <= data['open']) # Checks if open[0] is hit on the next candle
    data['target_high'] = (data_max_high >= data['high']) & (data_min_low <= data['high']) # Checks if high[0] is hit on the next candle
    data['target_low'] = (data_max_high >= data['low']) & (data_min_low <= data['low']) # Checks if low[0] is hit on the next candle

    data['target_open_1'] = (data_max_high >= data_backward['open']) & (data_min_low <= data_backward['open']) # Checks if previous open is hit on the next candle
    data['target_high_1'] = (data_max_high >= data_backward['high']) & (data_min_low <= data_backward['high']) # Checks if previous high is hit on the next candle
    data['target_low_1'] = (data_max_high >= data_backward['low']) & (data_min_low <= data_backward['low']) # Checks if previous low is hit on the next candle
    data['target_close_1'] = (data_max_high >= data_backward['close']) & (data_min_low <= data_backward['close']) # Checks if previous close is hit on the next candle

    # Get the statistics for each target
    stats = {
        'name' : [],
        'target' : [],
        'total' : [],
        'success' : [],
        'probability' : []
    } 

    for pattern in set(pattern_list):
        if pattern == -1:
            continue

        # Generate the series for that pattern
        array_pattern = np.array(pattern_list)
        array_pattern : np.array = array_pattern == pattern
        data['pattern'] = array_pattern
    
        for target in ["target_open", "target_high", "target_low", "target_open_1", "target_high_1", "target_low_1", "target_close_1"]:
            # Select rows where pattern is found
            selected = data[data['pattern']]
            count = len(selected)
            success = len(selected[selected[target]])
            probability = round(100 * success / count, 2)

            stats['name'].append(pattern)
            stats['target'].append(target)
            stats['total'].append(count)
            stats['success'].append(success)
            stats['probability'].append(probability)
            
    return pd.DataFrame(stats)
    

In [41]:
# analyse_pattern(engulfing, df, 3), analyse_pattern(bearish_engulfing, df, 3) 

In [42]:
# analyse_pattern(engulfing_strict_1, df, 3), analyse_pattern(bearish_engulfing_strict_1, df, 2)

In [43]:
# analyse_pattern(engulfing_strict_2, df, 3), analyse_pattern(bearish_engulfing_strict_2, df, 3)

In [44]:
# analyse_pattern(doji, df, 3), analyse_pattern(bearish_doji, df, 3)


In [45]:
# analyse_pattern(doji_strict, df, 3), analyse_pattern(bearish_doji_strict, df, 3) 

In [46]:
# analyse_pattern(inside, df, 3), analyse_pattern(bearish_inside, df, 3)

In [47]:
# analyse_pattern(kicker, df, 3), analyse_pattern(bearish_kicker, df, 3)

In [48]:
# analyse_pattern(piercing, df, 3), analyse_pattern(bearish_piercing, df, 3)

In [49]:
# analyse_pattern(piercing_strict, df, 3), analyse_pattern(bearish_piercing_strict, df, 3)

In [50]:
# analyse_pattern(piercing_strict_2, df, 3), analyse_pattern(bearish_piercing_strict_2, df)


### GENERIC CANDLE STICK TARGET STATISTICS
- Remake the dataframe into a directionless dataframe (columns : bmax, bmin)
- Generate dictionary of all patterns found

In [132]:
from itertools import combinations as iter_combinations

df_bland = df.copy()
df_bland['bmax'] = np.maximum(df_bland['open'], df_bland['close'])
df_bland['bmin'] = np.minimum(df_bland['open'], df_bland['close'])

In [53]:
df_bland

,open,high,low,close,hl2,bmax,bmin
date,,,,,,,
2000-09-18,1485.25,1489.75,1462.25,1467.50,1476.000,1485.25,1467.50
2000-09-19,1467.00,1482.75,1466.75,1478.50,1474.750,1478.50,1467.00
2000-09-20,1478.75,1480.50,1450.25,1469.50,1465.375,1478.75,1469.50
2000-09-21,1470.25,1474.00,1455.50,1469.50,1464.750,1470.25,1469.50
2000-09-22,1454.75,1471.00,1436.75,1468.50,1453.875,1468.50,1454.75
...,...,...,...,...,...,...,...
2024-05-16,5331.50,5349.00,5315.50,5320.25,5332.250,5331.50,5320.25
2024-05-17,5318.25,5328.75,5305.75,5327.25,5317.250,5327.25,5318.25
2024-05-20,5329.25,5348.25,5323.75,5331.75,5336.000,5331.75,5329.25


In [188]:
# Label the data points by candlestick pattern
all_patterns = {}

df_bland_columns = df_bland.columns.to_list()
df_bland_values = df_bland.values

n_candlesticks = 2 # How many candlesticks make a pattern
combination_length = 3 
comparison_indices = list(iter_combinations(range(n_candlesticks), 2))

# Initialize patterns_labels to match the length of df_bland
patterns_labels = [-1] * len(df_bland)

def compare_values(value_1, value_2, dimension=2):
    if dimension == 2:
        if value_1 > value_2:
            return 1
        else:
            return 0
    if dimension == 3:
        if value_1 > value_2:
            return 1
        elif value_1 < value_2:
            return -1
        else:
            return 0

for index in range(n_candlesticks, len(df_bland)):
    # Fetch bar data for the pattern
    data = df_bland_values[index - n_candlesticks: index]

    # Reverse the data : most recent bars come first
    data = data[::-1]
    
    # Convert each row into dictionaries
    data = [dict(zip(df_bland_columns, row)) for row in data]

    # Generate the pattern
    _patterns = []
    for indices in comparison_indices:
        data_A = data[indices[0]] # Current candle
        data_B = data[indices[1]] # Previous candle

        _pattern = [
            compare_values(data_A['high'], data_B['high']),
            compare_values(data_A['low'], data_B['low']),
            compare_values(data_A['open'], data_B['open']),
            compare_values(data_A['close'], data_B['close']),
            # compare_values(data_A['open'], data_B['close']),
            # compare_values(data_A['close'], data_B['open'])
        ]
        
        _patterns.append(tuple(_pattern))
       
    pattern_tuple = tuple(_patterns)
    all_patterns[pattern_tuple] = all_patterns.get(pattern_tuple, 0) + 1
    patterns_labels[index - 1] = list(all_patterns.keys()).index(pattern_tuple)


In [179]:
ccc = analyse_pattern_multi(patterns_labels, df_bland, 3)

In [180]:
selected = ccc[(ccc['total'] >= 100) & (ccc['probability'] >= 70)].sort_values(by='probability', ascending=False)
selected

,name,target,total,success,probability
71,10,target_high,115,105,91.30
70,10,target_open,115,102,88.70
36,5,target_high,500,435,87.00
76,10,target_close_1,115,100,86.96
92,13,target_high,228,198,86.84
106,15,target_high,104,90,86.54
43,6,target_high,1437,1243,86.50
34,4,target_close_1,502,427,85.06
28,4,target_open,502,424,84.46
109,15,target_high_1,104,87,83.65


In [183]:
list(all_patterns.keys())[10]

((0, 1, 1, 1),)

In [ ]:
patterns_labels

[-1,
 0,
 1,
 2,
 3,
 4,
 3,
 5,
 6,
 7,
 8,
 9,
 5,
 10,
 11,
 3,
 12,
 12,
 12,
 13,
 10,
 14,
 12,
 15,
 6,
 16,
 14,
 12,
 12,
 17,
 10,
 6,
 18,
 19,
 16,
 10,
 20,
 21,
 12,
 12,
 3,
 15,
 6,
 11,
 3,
 3,
 5,
 11,
 5,
 6,
 11,
 3,
 3,
 22,
 23,
 6,
 7,
 3,
 24,
 6,
 20,
 25,
 12,
 3,
 10,
 9,
 12,
 5,
 6,
 10,
 6,
 10,
 14,
 12,
 26,
 18,
 21,
 5,
 4,
 26,
 10,
 27,
 28,
 6,
 6,
 4,
 8,
 10,
 6,
 11,
 3,
 17,
 6,
 27,
 29,
 14,
 5,
 4,
 3,
 3,
 12,
 30,
 4,
 12,
 15,
 11,
 3,
 3,
 5,
 12,
 15,
 27,
 3,
 8,
 31,
 13,
 10,
 23,
 4,
 12,
 3,
 5,
 32,
 30,
 18,
 33,
 14,
 12,
 12,
 15,
 10,
 10,
 11,
 5,
 10,
 14,
 12,
 5,
 6,
 4,
 0,
 10,
 6,
 34,
 4,
 33,
 6,
 23,
 4,
 3,
 3,
 29,
 10,
 6,
 4,
 29,
 27,
 3,
 33,
 4,
 35,
 12,
 19,
 11,
 30,
 6,
 36,
 6,
 37,
 10,
 27,
 3,
 8,
 9,
 12,
 3,
 30,
 38,
 10,
 10,
 18,
 8,
 9,
 3,
 12,
 39,
 3,
 3,
 6,
 24,
 16,
 10,
 7,
 3,
 12,
 39,
 26,
 6,
 10,
 7,
 3,
 3,
 5,
 9,
 3,
 6,
 6,
 11,
 8,
 7,
 17,
 7,
 35,
 12,
 30,
 6,
 6,
 27,
 6,
 16,

In [ ]:
df_bland['labels'] = patterns_labels
df_bland[df_bland['labels'] == 33]

,open,high,low,close,hl2,bmax,bmin,labels
date,,,,,,,,
2001-03-19,1164.75,1186.75,1158.25,1180.50,1172.500,1180.50,1164.75,33
2001-04-17,1180.25,1200.75,1167.00,1195.50,1183.875,1195.50,1180.25,33
2001-05-04,1253.75,1274.00,1235.00,1272.75,1254.500,1272.75,1253.75,33
2001-10-10,1060.00,1085.25,1054.75,1083.25,1070.000,1083.25,1060.00,33
2002-01-15,1140.25,1150.00,1136.75,1148.50,1143.375,1148.50,1140.25,33
...,...,...,...,...,...,...,...,...
2023-05-10,4133.00,4173.25,4112.25,4152.00,4142.750,4152.00,4133.00,33
2023-07-17,4536.00,4565.75,4528.00,4553.75,4546.875,4553.75,4536.00,33
2023-07-24,4564.25,4592.50,4560.00,4583.50,4576.250,4583.50,4564.25,33


In [ ]:
ccc[(ccc['total'] >= 100) & (ccc['probability'] >= 60)]['target'].value_counts()

target
target_high       12
target_open_1     12
target_open       10
target_low         9
target_close_1     9
target_high_1      9
target_low_1       7
Name: count, dtype: int64

In [ ]:
[6,5,4,2,1,3].index(3)

5

In [197]:
def simulate_losing_streak(win_prob, rounds, simulations=10000):
    max_losing_streaks = []
    
    for _ in range(simulations):
        results = np.random.choice([0, 1], size=rounds, p=[1-win_prob, win_prob])
        current_streak = 0
        max_streak = 0
        
        for result in results:
            if result == 0:
                current_streak += 1
                if current_streak > max_streak:
                    max_streak = current_streak
            else:
                current_streak = 0
        
        max_losing_streaks.append(max_streak)
    
    return np.mean(max_losing_streaks)

# Example usage
win_prob = 0.7
rounds = 100

estimated_max_losing_streak = simulate_losing_streak(win_prob, rounds)
print(f"Estimated maximum losing streak: {estimated_max_losing_streak:.2f}")


Estimated maximum losing streak: 3.51
